In [ ]:
import sys
sys.path.append("..")

import Tools.Config as TC
import Tools.Func as TF
import Node.DS as DS
import time
import os

from pympler import asizeof
import re
import pandas as pd
dataset='webdocs'
total_count=0
heavy_ratio=1/5000

# =============================dataset path and file=============================
filepath=r"..\..\dataset\webdocs"
pattern='out_.*'
r=re.compile(pattern)
filelist=list(filter(r.match,os.listdir(filepath)))    
    # dataset file list

gr_file_name='webdocs_00_ground_truth.csv'        
gr_path=os.path.join(filepath,gr_file_name)
    # ground truth path
    
# =============================Initialize=============================
w=0
d=0
size=1033
topk=1033
TC.Set_default(w,d,size,topk)
    # set width, depth, size of Sk, random seed of hash
    # Config.width, Config.depth
Top_dict=dict()
    # Top_dict[x]=[count_x,error_x]

#item_count=100
# =============================Stream processing=============================
start=time.time()
for datafile in filelist[:1]:
    src_data=os.path.join(filepath,datafile)
    with open(src_data,'r') as file:
        while True:
            e=file.readline().strip('\n')
            if not e:
                print("EOF")
                break
            else:
                total_count+=1
                item=DS.ssNode(e,1)
                #item_count-=1
                # print("read {}th element: {}".format(item_count,element))
                if Top_dict.get(item.ID):
                    # e in Top
                    Top_dict[item.ID][0]+=item.count
                else:
                    if len(Top_dict)<TC.size:
                        Top_dict[item.ID]=[item.count,item.error]
                    else:
                        min_ele = min(Top_dict, key=lambda x: Top_dict.get(x)[0])
                            # find e_min
                        Top_dict[item.ID]=[Top_dict[min_ele][0]+item.count,Top_dict[min_ele][0]]
                            # update c_min,c_error
                        Top_dict.pop(min_ele)
                            # pop old min out
end=time.time()

# =============================Print and Plot result=============================
Top_dict=dict(sorted(Top_dict.items(), key=lambda item: item[1],reverse=True))
print("Top-{}".format(TC.size))
print("Execution time:{:8.3f} seconds.".format(end-start))

# =============================heavy hitter from result=============================
HH=dict()
for item in Top_dict:
    if Top_dict[item][0]>total_count*heavy_ratio:
        if (Top_dict[item][0]-Top_dict[item][1])>=total_count*heavy_ratio:
            HH[item]=Top_dict[item][0]
HH=dict(sorted(HH.items(), key=lambda item: item[1],reverse=True)) 

# memory usage
print("Top_dict with {} kbytes.".format(asizeof.asizeof(Top_dict)/1024))
# =============================heavy hitter from ground truth=============================

# read ground truth
import pandas as pd
df=pd.read_csv(gr_path)
df['Element']=df['Element'].astype('str')
temp=df[df['Count']>=int(total_count*heavy_ratio)]

gr_set=set(temp['Element'])
result=set(HH.keys())
tp_set=gr_set & result

print("\nFor copy")
print("Top-{}".format(TC.size))
print("Top_dict with {} kbytes.".format(asizeof.asizeof(Top_dict)/1024))
print("Find {:.3f} of Heavy Hitters".format(len(tp_set)/len(gr_set)))
print("Execution time:{:8.3f} seconds.".format(end-start))

# Count ARE/AAE in Top
startx=time.time()
top_are,top_aae=TF.Get_ARE_AAE(gr_path,HH,tp_set)
print("Find:{}".format(len(tp_set)))
#print("{} item found in SS[{}] compare with true Top-{}".format(len(tp_set),Config.size,topk))
endx=time.time()

print("Top_ARE: {:6.4f}".format(top_are))
print("Top_AAE: {:6.4f}".format(top_aae))
print("Estimate time:{:7.3f} seconds.".format(endx-startx)) 

TF.Plot_hh_compare(temp,HH,"SpaceSaving")

# result to csv
path="..\\result\\SS\\"+dataset+"\\HH\\Top_"+str(TC.size)
filename='SS_HH'+'_'+str(len(tp_set))+'.csv'
df=pd.DataFrame(Top_dict.items(),columns=['Element','Count'])
df['Element'] = df['Element'].astype(str)
df.to_csv(os.path.join(path,filename),index=False)

In [ ]:
import sys
sys.path.append("..")

import Tools.Config as TC
import Tools.Func as TF
import Node.DS as DS
import time
import os

from pympler import asizeof
import re
import pandas as pd
dataset='webdocs'
total_count=0
heavy_ratio=1/5000

# =============================dataset path and file=============================
filepath=r"..\..\dataset\webdocs"
pattern='out_.*'
r=re.compile(pattern)
filelist=list(filter(r.match,os.listdir(filepath)))    
    # dataset file list

gr_file_name='webdocs_00_ground_truth.csv'        
gr_path=os.path.join(filepath,gr_file_name)
    # ground truth path
    
# =============================Initialize=============================
w=0
d=0
size=1500
topk=1500
TC.Set_default(w,d,size,topk)
    # set width, depth, size of Sk, random seed of hash
    # Config.width, Config.depth
Top_dict=dict()
    # Top_dict[x]=[count_x,error_x]

#item_count=100
# =============================Stream processing=============================
start=time.time()
for datafile in filelist[:1]:
    src_data=os.path.join(filepath,datafile)
    with open(src_data,'r') as file:
        while True:
            e=file.readline().strip('\n')
            if not e:
                print("EOF")
                break
            else:
                total_count+=1
                item=DS.ssNode(e,1)
                #item_count-=1
                # print("read {}th element: {}".format(item_count,element))
                if Top_dict.get(item.ID):
                    # e in Top
                    Top_dict[item.ID][0]+=item.count
                else:
                    if len(Top_dict)<TC.size:
                        Top_dict[item.ID]=[item.count,item.error]
                    else:
                        min_ele = min(Top_dict, key=lambda x: Top_dict.get(x)[0])
                            # find e_min
                        Top_dict[item.ID]=[Top_dict[min_ele][0]+item.count,Top_dict[min_ele][0]]
                            # update c_min,c_error
                        Top_dict.pop(min_ele)
                            # pop old min out
end=time.time()

# =============================Print and Plot result=============================
Top_dict=dict(sorted(Top_dict.items(), key=lambda item: item[1],reverse=True))
print("Top-{}".format(TC.size))
print("Execution time:{:8.3f} seconds.".format(end-start))

# =============================heavy hitter from result=============================
HH=dict()
for item in Top_dict:
    if Top_dict[item][0]>total_count*heavy_ratio:
        if (Top_dict[item][0]-Top_dict[item][1])>=total_count*heavy_ratio:
            HH[item]=Top_dict[item][0]
HH=dict(sorted(HH.items(), key=lambda item: item[1],reverse=True)) 

# memory usage
print("Top_dict with {} kbytes.".format(asizeof.asizeof(Top_dict)/1024))
# =============================heavy hitter from ground truth=============================

# read ground truth
import pandas as pd
df=pd.read_csv(gr_path)
df['Element']=df['Element'].astype('str')
temp=df[df['Count']>=int(total_count*heavy_ratio)]

gr_set=set(temp['Element'])
result=set(HH.keys())
tp_set=gr_set & result

print("\nFor copy")
print("Top-{}".format(TC.size))
print("Top_dict with {} kbytes.".format(asizeof.asizeof(Top_dict)/1024))
print("Find {:.3f} of Heavy Hitters".format(len(tp_set)/len(gr_set)))
print("Execution time:{:8.3f} seconds.".format(end-start))

# Count ARE/AAE in Top
startx=time.time()
top_are,top_aae=TF.Get_ARE_AAE(gr_path,HH,tp_set)
print("Find:{}".format(len(tp_set)))
#print("{} item found in SS[{}] compare with true Top-{}".format(len(tp_set),Config.size,topk))
endx=time.time()

print("Top_ARE: {:6.4f}".format(top_are))
print("Top_AAE: {:6.4f}".format(top_aae))
print("Estimate time:{:7.3f} seconds.".format(endx-startx)) 

TF.Plot_hh_compare(temp,HH,"SpaceSaving")

# result to csv
path="..\\result\\SS\\"+dataset+"\\HH\\Top_"+str(TC.size)
filename='SS_HH'+'_'+str(len(tp_set))+'.csv'
df=pd.DataFrame(Top_dict.items(),columns=['Element','Count'])
df['Element'] = df['Element'].astype(str)
df.to_csv(os.path.join(path,filename),index=False)

In [ ]:
import sys
sys.path.append("..")

import Tools.Config as TC
import Tools.Func as TF
import Node.DS as DS
import time
import os

from pympler import asizeof
import re
import pandas as pd
dataset='webdocs'
total_count=0
heavy_ratio=1/5000

# =============================dataset path and file=============================
filepath=r"..\..\dataset\webdocs"
pattern='out_.*'
r=re.compile(pattern)
filelist=list(filter(r.match,os.listdir(filepath)))    
    # dataset file list

gr_file_name='webdocs_00_ground_truth.csv'        
gr_path=os.path.join(filepath,gr_file_name)
    # ground truth path
    
# =============================Initialize=============================
w=0
d=0
size=5000
topk=5000
TC.Set_default(w,d,size,topk)
    # set width, depth, size of Sk, random seed of hash
    # Config.width, Config.depth
Top_dict=dict()
    # Top_dict[x]=[count_x,error_x]

#item_count=100
# =============================Stream processing=============================
start=time.time()
for datafile in filelist[:1]:
    src_data=os.path.join(filepath,datafile)
    with open(src_data,'r') as file:
        while True:
            e=file.readline().strip('\n')
            if not e:
                print("EOF")
                break
            else:
                total_count+=1
                item=DS.ssNode(e,1)
                #item_count-=1
                # print("read {}th element: {}".format(item_count,element))
                if Top_dict.get(item.ID):
                    # e in Top
                    Top_dict[item.ID][0]+=item.count
                else:
                    if len(Top_dict)<TC.size:
                        Top_dict[item.ID]=[item.count,item.error]
                    else:
                        min_ele = min(Top_dict, key=lambda x: Top_dict.get(x)[0])
                            # find e_min
                        Top_dict[item.ID]=[Top_dict[min_ele][0]+item.count,Top_dict[min_ele][0]]
                            # update c_min,c_error
                        Top_dict.pop(min_ele)
                            # pop old min out
end=time.time()

# =============================Print and Plot result=============================
Top_dict=dict(sorted(Top_dict.items(), key=lambda item: item[1],reverse=True))
print("Top-{}".format(TC.size))
print("Execution time:{:8.3f} seconds.".format(end-start))

# =============================heavy hitter from result=============================
HH=dict()
for item in Top_dict:
    if Top_dict[item][0]>total_count*heavy_ratio:
        if (Top_dict[item][0]-Top_dict[item][1])>=total_count*heavy_ratio:
            HH[item]=Top_dict[item][0]
HH=dict(sorted(HH.items(), key=lambda item: item[1],reverse=True)) 

# memory usage
print("Top_dict with {} kbytes.".format(asizeof.asizeof(Top_dict)/1024))
# =============================heavy hitter from ground truth=============================

# read ground truth
import pandas as pd
df=pd.read_csv(gr_path)
df['Element']=df['Element'].astype('str')
temp=df[df['Count']>=int(total_count*heavy_ratio)]

gr_set=set(temp['Element'])
result=set(HH.keys())
tp_set=gr_set & result

print("\nFor copy")
print("Top-{}".format(TC.size))
print("Top_dict with {} kbytes.".format(asizeof.asizeof(Top_dict)/1024))
print("Find {:.3f} of Heavy Hitters".format(len(tp_set)/len(gr_set)))
print("Execution time:{:8.3f} seconds.".format(end-start))

# Count ARE/AAE in Top
startx=time.time()
top_are,top_aae=TF.Get_ARE_AAE(gr_path,HH,tp_set)
print("Find:{}".format(len(tp_set)))
#print("{} item found in SS[{}] compare with true Top-{}".format(len(tp_set),Config.size,topk))
endx=time.time()

print("Top_ARE: {:6.4f}".format(top_are))
print("Top_AAE: {:6.4f}".format(top_aae))
print("Estimate time:{:7.3f} seconds.".format(endx-startx)) 

TF.Plot_hh_compare(temp,HH,"SpaceSaving")


# result to csv
path="..\\result\\SS\\"+dataset+"\\HH\\Top_"+str(TC.size)
filename='SS_HH'+'_'+str(len(tp_set))+'.csv'
df=pd.DataFrame(Top_dict.items(),columns=['Element','Count'])
df['Element'] = df['Element'].astype(str)
df.to_csv(os.path.join(path,filename),index=False)

In [ ]:
import sys
sys.path.append("..")

import Tools.Config as TC
import Tools.Func as TF
import Node.DS as DS
import time
import os

from pympler import asizeof
import re
import pandas as pd
dataset='webdocs'
total_count=0
heavy_ratio=1/5000

# =============================dataset path and file=============================
filepath=r"..\..\dataset\webdocs"
pattern='out_.*'
r=re.compile(pattern)
filelist=list(filter(r.match,os.listdir(filepath)))    
    # dataset file list

gr_file_name='webdocs_00_ground_truth.csv'        
gr_path=os.path.join(filepath,gr_file_name)
    # ground truth path
    
# =============================Initialize=============================
w=0
d=0
size=4500
topk=4500
TC.Set_default(w,d,size,topk)
    # set width, depth, size of Sk, random seed of hash
    # Config.width, Config.depth
Top_dict=dict()
    # Top_dict[x]=[count_x,error_x]

#item_count=100
# =============================Stream processing=============================
start=time.time()
for datafile in filelist[:1]:
    src_data=os.path.join(filepath,datafile)
    with open(src_data,'r') as file:
        while True:
            e=file.readline().strip('\n')
            if not e:
                print("EOF")
                break
            else:
                total_count+=1
                item=DS.ssNode(e,1)
                #item_count-=1
                # print("read {}th element: {}".format(item_count,element))
                if Top_dict.get(item.ID):
                    # e in Top
                    Top_dict[item.ID][0]+=item.count
                else:
                    if len(Top_dict)<TC.size:
                        Top_dict[item.ID]=[item.count,item.error]
                    else:
                        min_ele = min(Top_dict, key=lambda x: Top_dict.get(x)[0])
                            # find e_min
                        Top_dict[item.ID]=[Top_dict[min_ele][0]+item.count,Top_dict[min_ele][0]]
                            # update c_min,c_error
                        Top_dict.pop(min_ele)
                            # pop old min out
end=time.time()

# =============================Print and Plot result=============================
Top_dict=dict(sorted(Top_dict.items(), key=lambda item: item[1],reverse=True))
print("Top-{}".format(TC.size))
print("Execution time:{:8.3f} seconds.".format(end-start))

# =============================heavy hitter from result=============================
HH=dict()
for item in Top_dict:
    if Top_dict[item][0]>total_count*heavy_ratio:
        if (Top_dict[item][0]-Top_dict[item][1])>=total_count*heavy_ratio:
            HH[item]=Top_dict[item][0]
HH=dict(sorted(HH.items(), key=lambda item: item[1],reverse=True)) 

# memory usage
print("Top_dict with {} kbytes.".format(asizeof.asizeof(Top_dict)/1024))
# =============================heavy hitter from ground truth=============================

# read ground truth
import pandas as pd
df=pd.read_csv(gr_path)
df['Element']=df['Element'].astype('str')
temp=df[df['Count']>=int(total_count*heavy_ratio)]

gr_set=set(temp['Element'])
result=set(HH.keys())
tp_set=gr_set & result

print("\nFor copy")
print("Top-{}".format(TC.size))
print("Top_dict with {} kbytes.".format(asizeof.asizeof(Top_dict)/1024))
print("Find {:.3f} of Heavy Hitters".format(len(tp_set)/len(gr_set)))
print("Execution time:{:8.3f} seconds.".format(end-start))

# Count ARE/AAE in Top
startx=time.time()
top_are,top_aae=TF.Get_ARE_AAE(gr_path,HH,tp_set)
print("Find:{}".format(len(tp_set)))
#print("{} item found in SS[{}] compare with true Top-{}".format(len(tp_set),Config.size,topk))
endx=time.time()

print("Top_ARE: {:6.4f}".format(top_are))
print("Top_AAE: {:6.4f}".format(top_aae))
print("Estimate time:{:7.3f} seconds.".format(endx-startx)) 

TF.Plot_hh_compare(temp,HH,"SpaceSaving")

# result to csv
path="..\\result\\SS\\"+dataset+"\\HH\\Top_"+str(TC.size)
filename='SS_HH'+'_'+str(len(tp_set))+'.csv'
df=pd.DataFrame(Top_dict.items(),columns=['Element','Count'])
df['Element'] = df['Element'].astype(str)
df.to_csv(os.path.join(path,filename),index=False)

In [ ]:
import sys
sys.path.append("..")

import Tools.Config as TC
import Tools.Func as TF
import Node.DS as DS
import time
import os

from pympler import asizeof
import re
import pandas as pd
dataset='webdocs'
total_count=0
heavy_ratio=1/5000

# =============================dataset path and file=============================
filepath=r"..\..\dataset\webdocs"
pattern='out_.*'
r=re.compile(pattern)
filelist=list(filter(r.match,os.listdir(filepath)))    
    # dataset file list

gr_file_name='webdocs_00_ground_truth.csv'        
gr_path=os.path.join(filepath,gr_file_name)
    # ground truth path
    
# =============================Initialize=============================
w=0
d=0
size=4000
topk=4000
TC.Set_default(w,d,size,topk)
    # set width, depth, size of Sk, random seed of hash
    # Config.width, Config.depth
Top_dict=dict()
    # Top_dict[x]=[count_x,error_x]

#item_count=100
# =============================Stream processing=============================
start=time.time()
for datafile in filelist[:1]:
    src_data=os.path.join(filepath,datafile)
    with open(src_data,'r') as file:
        while True:
            e=file.readline().strip('\n')
            if not e:
                print("EOF")
                break
            else:
                total_count+=1
                item=DS.ssNode(e,1)
                #item_count-=1
                # print("read {}th element: {}".format(item_count,element))
                if Top_dict.get(item.ID):
                    # e in Top
                    Top_dict[item.ID][0]+=item.count
                else:
                    if len(Top_dict)<TC.size:
                        Top_dict[item.ID]=[item.count,item.error]
                    else:
                        min_ele = min(Top_dict, key=lambda x: Top_dict.get(x)[0])
                            # find e_min
                        Top_dict[item.ID]=[Top_dict[min_ele][0]+item.count,Top_dict[min_ele][0]]
                            # update c_min,c_error
                        Top_dict.pop(min_ele)
                            # pop old min out
end=time.time()

# =============================Print and Plot result=============================
Top_dict=dict(sorted(Top_dict.items(), key=lambda item: item[1],reverse=True))
print("Top-{}".format(TC.size))
print("Execution time:{:8.3f} seconds.".format(end-start))

# =============================heavy hitter from result=============================
HH=dict()
for item in Top_dict:
    if Top_dict[item][0]>total_count*heavy_ratio:
        if (Top_dict[item][0]-Top_dict[item][1])>=total_count*heavy_ratio:
            HH[item]=Top_dict[item][0]
HH=dict(sorted(HH.items(), key=lambda item: item[1],reverse=True)) 

# memory usage
print("Top_dict with {} kbytes.".format(asizeof.asizeof(Top_dict)/1024))
# =============================heavy hitter from ground truth=============================

# read ground truth
import pandas as pd
df=pd.read_csv(gr_path)
df['Element']=df['Element'].astype('str')
temp=df[df['Count']>=int(total_count*heavy_ratio)]

gr_set=set(temp['Element'])
result=set(HH.keys())
tp_set=gr_set & result

print("\nFor copy")
print("Top-{}".format(TC.size))
print("Top_dict with {} kbytes.".format(asizeof.asizeof(Top_dict)/1024))
print("Find {:.3f} of Heavy Hitters".format(len(tp_set)/len(gr_set)))
print("Execution time:{:8.3f} seconds.".format(end-start))

# Count ARE/AAE in Top
startx=time.time()
top_are,top_aae=TF.Get_ARE_AAE(gr_path,HH,tp_set)
print("Find:{}".format(len(tp_set)))
#print("{} item found in SS[{}] compare with true Top-{}".format(len(tp_set),Config.size,topk))
endx=time.time()

print("Top_ARE: {:6.4f}".format(top_are))
print("Top_AAE: {:6.4f}".format(top_aae))
print("Estimate time:{:7.3f} seconds.".format(endx-startx)) 

TF.Plot_hh_compare(temp,HH,"SpaceSaving")

# result to csv
path="..\\result\\SS\\"+dataset+"\\HH\\Top_"+str(TC.size)
filename='SS_HH'+'_'+str(len(tp_set))+'.csv'
df=pd.DataFrame(Top_dict.items(),columns=['Element','Count'])
df['Element'] = df['Element'].astype(str)
df.to_csv(os.path.join(path,filename),index=False)

In [ ]:
import sys
sys.path.append("..")

import Tools.Config as TC
import Tools.Func as TF
import Node.DS as DS
import time
import os

from pympler import asizeof
import re
import pandas as pd
dataset='webdocs'
total_count=0
heavy_ratio=1/5000

# =============================dataset path and file=============================
filepath=r"..\..\dataset\webdocs"
pattern='out_.*'
r=re.compile(pattern)
filelist=list(filter(r.match,os.listdir(filepath)))    
    # dataset file list

gr_file_name='webdocs_00_ground_truth.csv'        
gr_path=os.path.join(filepath,gr_file_name)
    # ground truth path
    
# =============================Initialize=============================
w=0
d=0
size=3500
topk=3500
TC.Set_default(w,d,size,topk)
    # set width, depth, size of Sk, random seed of hash
    # Config.width, Config.depth
Top_dict=dict()
    # Top_dict[x]=[count_x,error_x]

#item_count=100
# =============================Stream processing=============================
start=time.time()
for datafile in filelist[:1]:
    src_data=os.path.join(filepath,datafile)
    with open(src_data,'r') as file:
        while True:
            e=file.readline().strip('\n')
            if not e:
                print("EOF")
                break
            else:
                total_count+=1
                item=DS.ssNode(e,1)
                #item_count-=1
                # print("read {}th element: {}".format(item_count,element))
                if Top_dict.get(item.ID):
                    # e in Top
                    Top_dict[item.ID][0]+=item.count
                else:
                    if len(Top_dict)<TC.size:
                        Top_dict[item.ID]=[item.count,item.error]
                    else:
                        min_ele = min(Top_dict, key=lambda x: Top_dict.get(x)[0])
                            # find e_min
                        Top_dict[item.ID]=[Top_dict[min_ele][0]+item.count,Top_dict[min_ele][0]]
                            # update c_min,c_error
                        Top_dict.pop(min_ele)
                            # pop old min out
end=time.time()

# =============================Print and Plot result=============================
Top_dict=dict(sorted(Top_dict.items(), key=lambda item: item[1],reverse=True))
print("Top-{}".format(TC.size))
print("Execution time:{:8.3f} seconds.".format(end-start))

# =============================heavy hitter from result=============================
HH=dict()
for item in Top_dict:
    if Top_dict[item][0]>total_count*heavy_ratio:
        if (Top_dict[item][0]-Top_dict[item][1])>=total_count*heavy_ratio:
            HH[item]=Top_dict[item][0]
HH=dict(sorted(HH.items(), key=lambda item: item[1],reverse=True)) 

# memory usage
print("Top_dict with {} kbytes.".format(asizeof.asizeof(Top_dict)/1024))
# =============================heavy hitter from ground truth=============================

# read ground truth
import pandas as pd
df=pd.read_csv(gr_path)
df['Element']=df['Element'].astype('str')
temp=df[df['Count']>=int(total_count*heavy_ratio)]

gr_set=set(temp['Element'])
result=set(HH.keys())
tp_set=gr_set & result

print("\nFor copy")
print("Top-{}".format(TC.size))
print("Top_dict with {} kbytes.".format(asizeof.asizeof(Top_dict)/1024))
print("Find {:.3f} of Heavy Hitters".format(len(tp_set)/len(gr_set)))
print("Execution time:{:8.3f} seconds.".format(end-start))

# Count ARE/AAE in Top
startx=time.time()
top_are,top_aae=TF.Get_ARE_AAE(gr_path,HH,tp_set)
print("Find:{}".format(len(tp_set)))
#print("{} item found in SS[{}] compare with true Top-{}".format(len(tp_set),Config.size,topk))
endx=time.time()

print("Top_ARE: {:6.4f}".format(top_are))
print("Top_AAE: {:6.4f}".format(top_aae))
print("Estimate time:{:7.3f} seconds.".format(endx-startx)) 

TF.Plot_hh_compare(temp,HH,"SpaceSaving")

# result to csv
path="..\\result\\SS\\"+dataset+"\\HH\\Top_"+str(TC.size)
filename='SS_HH'+'_'+str(len(tp_set))+'.csv'
df=pd.DataFrame(Top_dict.items(),columns=['Element','Count'])
df['Element'] = df['Element'].astype(str)
df.to_csv(os.path.join(path,filename),index=False)

In [ ]:
import sys
sys.path.append("..")

import Tools.Config as TC
import Tools.Func as TF
import Node.DS as DS
import time
import os

from pympler import asizeof
import re
import pandas as pd
dataset='webdocs'
total_count=0
heavy_ratio=1/5000

# =============================dataset path and file=============================
filepath=r"..\..\dataset\webdocs"
pattern='out_.*'
r=re.compile(pattern)
filelist=list(filter(r.match,os.listdir(filepath)))    
    # dataset file list

gr_file_name='webdocs_00_ground_truth.csv'        
gr_path=os.path.join(filepath,gr_file_name)
    # ground truth path
    
# =============================Initialize=============================
w=0
d=0
size=3000
topk=3000
TC.Set_default(w,d,size,topk)
    # set width, depth, size of Sk, random seed of hash
    # Config.width, Config.depth
Top_dict=dict()
    # Top_dict[x]=[count_x,error_x]

#item_count=100
# =============================Stream processing=============================
start=time.time()
for datafile in filelist[:1]:
    src_data=os.path.join(filepath,datafile)
    with open(src_data,'r') as file:
        while True:
            e=file.readline().strip('\n')
            if not e:
                print("EOF")
                break
            else:
                total_count+=1
                item=DS.ssNode(e,1)
                #item_count-=1
                # print("read {}th element: {}".format(item_count,element))
                if Top_dict.get(item.ID):
                    # e in Top
                    Top_dict[item.ID][0]+=item.count
                else:
                    if len(Top_dict)<TC.size:
                        Top_dict[item.ID]=[item.count,item.error]
                    else:
                        min_ele = min(Top_dict, key=lambda x: Top_dict.get(x)[0])
                            # find e_min
                        Top_dict[item.ID]=[Top_dict[min_ele][0]+item.count,Top_dict[min_ele][0]]
                            # update c_min,c_error
                        Top_dict.pop(min_ele)
                            # pop old min out
end=time.time()

# =============================Print and Plot result=============================
Top_dict=dict(sorted(Top_dict.items(), key=lambda item: item[1],reverse=True))
print("Top-{}".format(TC.size))
print("Execution time:{:8.3f} seconds.".format(end-start))

# =============================heavy hitter from result=============================
HH=dict()
for item in Top_dict:
    if Top_dict[item][0]>total_count*heavy_ratio:
        if (Top_dict[item][0]-Top_dict[item][1])>=total_count*heavy_ratio:
            HH[item]=Top_dict[item][0]
HH=dict(sorted(HH.items(), key=lambda item: item[1],reverse=True)) 

# memory usage
print("Top_dict with {} kbytes.".format(asizeof.asizeof(Top_dict)/1024))
# =============================heavy hitter from ground truth=============================

# read ground truth
import pandas as pd
df=pd.read_csv(gr_path)
df['Element']=df['Element'].astype('str')
temp=df[df['Count']>=int(total_count*heavy_ratio)]

gr_set=set(temp['Element'])
result=set(HH.keys())
tp_set=gr_set & result

print("\nFor copy")
print("Top-{}".format(TC.size))
print("Top_dict with {} kbytes.".format(asizeof.asizeof(Top_dict)/1024))
print("Find {:.3f} of Heavy Hitters".format(len(tp_set)/len(gr_set)))
print("Execution time:{:8.3f} seconds.".format(end-start))

# Count ARE/AAE in Top
startx=time.time()
top_are,top_aae=TF.Get_ARE_AAE(gr_path,HH,tp_set)
print("Find:{}".format(len(tp_set)))
#print("{} item found in SS[{}] compare with true Top-{}".format(len(tp_set),Config.size,topk))
endx=time.time()

print("Top_ARE: {:6.4f}".format(top_are))
print("Top_AAE: {:6.4f}".format(top_aae))
print("Estimate time:{:7.3f} seconds.".format(endx-startx)) 

TF.Plot_hh_compare(temp,HH,"SpaceSaving")

# result to csv
path="..\\result\\SS\\"+dataset+"\\HH\\Top_"+str(TC.size)
filename='SS_HH'+'_'+str(len(tp_set))+'.csv'
df=pd.DataFrame(Top_dict.items(),columns=['Element','Count'])
df['Element'] = df['Element'].astype(str)
df.to_csv(os.path.join(path,filename),index=False)

In [ ]:
import sys
sys.path.append("..")

import Tools.Config as TC
import Tools.Func as TF
import Node.DS as DS
import time
import os

from pympler import asizeof
import re
import pandas as pd
dataset='webdocs'
total_count=0
heavy_ratio=1/5000

# =============================dataset path and file=============================
filepath=r"..\..\dataset\webdocs"
pattern='out_.*'
r=re.compile(pattern)
filelist=list(filter(r.match,os.listdir(filepath)))    
    # dataset file list

gr_file_name='webdocs_00_ground_truth.csv'        
gr_path=os.path.join(filepath,gr_file_name)
    # ground truth path
    
# =============================Initialize=============================
w=0
d=0
size=2500
topk=2500
TC.Set_default(w,d,size,topk)
    # set width, depth, size of Sk, random seed of hash
    # Config.width, Config.depth
Top_dict=dict()
    # Top_dict[x]=[count_x,error_x]

#item_count=100
# =============================Stream processing=============================
start=time.time()
for datafile in filelist[:1]:
    src_data=os.path.join(filepath,datafile)
    with open(src_data,'r') as file:
        while True:
            e=file.readline().strip('\n')
            if not e:
                print("EOF")
                break
            else:
                total_count+=1
                item=DS.ssNode(e,1)
                #item_count-=1
                # print("read {}th element: {}".format(item_count,element))
                if Top_dict.get(item.ID):
                    # e in Top
                    Top_dict[item.ID][0]+=item.count
                else:
                    if len(Top_dict)<TC.size:
                        Top_dict[item.ID]=[item.count,item.error]
                    else:
                        min_ele = min(Top_dict, key=lambda x: Top_dict.get(x)[0])
                            # find e_min
                        Top_dict[item.ID]=[Top_dict[min_ele][0]+item.count,Top_dict[min_ele][0]]
                            # update c_min,c_error
                        Top_dict.pop(min_ele)
                            # pop old min out
end=time.time()

# =============================Print and Plot result=============================
Top_dict=dict(sorted(Top_dict.items(), key=lambda item: item[1],reverse=True))
print("Top-{}".format(TC.size))
print("Execution time:{:8.3f} seconds.".format(end-start))

# =============================heavy hitter from result=============================
HH=dict()
for item in Top_dict:
    if Top_dict[item][0]>total_count*heavy_ratio:
        if (Top_dict[item][0]-Top_dict[item][1])>=total_count*heavy_ratio:
            HH[item]=Top_dict[item][0]
HH=dict(sorted(HH.items(), key=lambda item: item[1],reverse=True)) 

# memory usage
print("Top_dict with {} kbytes.".format(asizeof.asizeof(Top_dict)/1024))
# =============================heavy hitter from ground truth=============================

# read ground truth
import pandas as pd
df=pd.read_csv(gr_path)
df['Element']=df['Element'].astype('str')
temp=df[df['Count']>=int(total_count*heavy_ratio)]

gr_set=set(temp['Element'])
result=set(HH.keys())
tp_set=gr_set & result

print("\nFor copy")
print("Top-{}".format(TC.size))
print("Top_dict with {} kbytes.".format(asizeof.asizeof(Top_dict)/1024))
print("Find {:.3f} of Heavy Hitters".format(len(tp_set)/len(gr_set)))
print("Execution time:{:8.3f} seconds.".format(end-start))

# Count ARE/AAE in Top
startx=time.time()
top_are,top_aae=TF.Get_ARE_AAE(gr_path,HH,tp_set)
print("Find:{}".format(len(tp_set)))
#print("{} item found in SS[{}] compare with true Top-{}".format(len(tp_set),Config.size,topk))
endx=time.time()

print("Top_ARE: {:6.4f}".format(top_are))
print("Top_AAE: {:6.4f}".format(top_aae))
print("Estimate time:{:7.3f} seconds.".format(endx-startx)) 

TF.Plot_hh_compare(temp,HH,"SpaceSaving")

# result to csv
path="..\\result\\SS\\"+dataset+"\\HH\\Top_"+str(TC.size)
filename='SS_HH'+'_'+str(len(tp_set))+'.csv'
df=pd.DataFrame(Top_dict.items(),columns=['Element','Count'])
df['Element'] = df['Element'].astype(str)
df.to_csv(os.path.join(path,filename),index=False)

In [ ]:
import sys
sys.path.append("..")

import Tools.Config as TC
import Tools.Func as TF
import Node.DS as DS
import time
import os

from pympler import asizeof
import re
import pandas as pd
dataset='webdocs'
total_count=0
heavy_ratio=1/5000

# =============================dataset path and file=============================
filepath=r"..\..\dataset\webdocs"
pattern='out_.*'
r=re.compile(pattern)
filelist=list(filter(r.match,os.listdir(filepath)))    
    # dataset file list

gr_file_name='webdocs_00_ground_truth.csv'        
gr_path=os.path.join(filepath,gr_file_name)
    # ground truth path
    
# =============================Initialize=============================
w=0
d=0
size=2000
topk=2000
TC.Set_default(w,d,size,topk)
    # set width, depth, size of Sk, random seed of hash
    # Config.width, Config.depth
Top_dict=dict()
    # Top_dict[x]=[count_x,error_x]

#item_count=100
# =============================Stream processing=============================
start=time.time()
for datafile in filelist[:1]:
    src_data=os.path.join(filepath,datafile)
    with open(src_data,'r') as file:
        while True:
            e=file.readline().strip('\n')
            if not e:
                print("EOF")
                break
            else:
                total_count+=1
                item=DS.ssNode(e,1)
                #item_count-=1
                # print("read {}th element: {}".format(item_count,element))
                if Top_dict.get(item.ID):
                    # e in Top
                    Top_dict[item.ID][0]+=item.count
                else:
                    if len(Top_dict)<TC.size:
                        Top_dict[item.ID]=[item.count,item.error]
                    else:
                        min_ele = min(Top_dict, key=lambda x: Top_dict.get(x)[0])
                            # find e_min
                        Top_dict[item.ID]=[Top_dict[min_ele][0]+item.count,Top_dict[min_ele][0]]
                            # update c_min,c_error
                        Top_dict.pop(min_ele)
                            # pop old min out
end=time.time()

# =============================Print and Plot result=============================
Top_dict=dict(sorted(Top_dict.items(), key=lambda item: item[1],reverse=True))
print("Top-{}".format(TC.size))
print("Execution time:{:8.3f} seconds.".format(end-start))

# =============================heavy hitter from result=============================
HH=dict()
for item in Top_dict:
    if Top_dict[item][0]>total_count*heavy_ratio:
        if (Top_dict[item][0]-Top_dict[item][1])>=total_count*heavy_ratio:
            HH[item]=Top_dict[item][0]
HH=dict(sorted(HH.items(), key=lambda item: item[1],reverse=True)) 

# memory usage
print("Top_dict with {} kbytes.".format(asizeof.asizeof(Top_dict)/1024))
# =============================heavy hitter from ground truth=============================

# read ground truth
import pandas as pd
df=pd.read_csv(gr_path)
df['Element']=df['Element'].astype('str')
temp=df[df['Count']>=int(total_count*heavy_ratio)]

gr_set=set(temp['Element'])
result=set(HH.keys())
tp_set=gr_set & result

print("\nFor copy")
print("Top-{}".format(TC.size))
print("Top_dict with {} kbytes.".format(asizeof.asizeof(Top_dict)/1024))
print("Find {:.3f} of Heavy Hitters".format(len(tp_set)/len(gr_set)))
print("Execution time:{:8.3f} seconds.".format(end-start))

# Count ARE/AAE in Top
startx=time.time()
top_are,top_aae=TF.Get_ARE_AAE(gr_path,HH,tp_set)
print("Find:{}".format(len(tp_set)))
#print("{} item found in SS[{}] compare with true Top-{}".format(len(tp_set),Config.size,topk))
endx=time.time()

print("Top_ARE: {:6.4f}".format(top_are))
print("Top_AAE: {:6.4f}".format(top_aae))
print("Estimate time:{:7.3f} seconds.".format(endx-startx)) 

TF.Plot_hh_compare(temp,HH,"SpaceSaving")

# result to csv
path="..\\result\\SS\\"+dataset+"\\HH\\Top_"+str(TC.size)
filename='SS_HH'+'_'+str(len(tp_set))+'.csv'
df=pd.DataFrame(Top_dict.items(),columns=['Element','Count'])
df['Element'] = df['Element'].astype(str)
df.to_csv(os.path.join(path,filename),index=False)

In [ ]:
import numpy as np
x=np.zeros(2,dtype='int32')
x

In [2]:
import sys
sys.path.append("..")

import Tools.Config as TC
import Tools.Func as TF
import Node.DS as DS
import time
import os

from pympler import asizeof
import re
import pandas as pd
import numpy as np
dataset='webdocs'
total_count=0
heavy_ratio=1/5000

# =============================dataset path and file=============================
filepath=r"..\..\dataset\webdocs"
pattern='out_.*'
r=re.compile(pattern)
filelist=list(filter(r.match,os.listdir(filepath)))    
    # dataset file list

gr_file_name='webdocs_00_ground_truth.csv'        
gr_path=os.path.join(filepath,gr_file_name)
    # ground truth path
    
# =============================Initialize=============================
w=0
d=0
size=5
topk=5
TC.Set_default(w,d,size,topk)
    # set width, depth, size of Sk, random seed of hash
    # Config.width, Config.depth
Top_dict=dict()
    # Top_dict[x]=[count_x,error_x]

item_count=10

# =============================Stream processing=============================
start=time.time()
for datafile in filelist[:1]:
    src_data=os.path.join(filepath,datafile)
    with open(src_data,'r') as file:
        while item_count:
            item=file.readline().strip('\n')
            if not item:
                print("EOF")
                break
            else:
                print("Incoming:{}".format(item))
                print("Top_dict:{}".format(Top_dict))
                total_count+=1
                item_count-=1
                # print("read {}th element: {}".format(item_count,element))
                if Top_dict.get(item):
                    # e in Top
                    Top_dict[item][0]+=1
                else:
                    if len(Top_dict)<TC.size:
                        Top_dict[item]=np.array((1,0),dtype='int32')
                    else:
                        min_ele = min(Top_dict, key=lambda x: Top_dict.get(x)[0])
                            # find e_min
                        Top_dict[item]=[Top_dict[min_ele][0]+1,Top_dict[min_ele][0]]
                            # update c_min,c_error
                        Top_dict.pop(min_ele)
                            # pop old min out
end=time.time()


Incoming:1
Top_dict:{}
Incoming:2
Top_dict:{'1': array([1, 0])}
Incoming:3
Top_dict:{'1': array([1, 0]), '2': array([1, 0])}
Incoming:4
Top_dict:{'1': array([1, 0]), '2': array([1, 0]), '3': array([1, 0])}
Incoming:5
Top_dict:{'1': array([1, 0]), '2': array([1, 0]), '3': array([1, 0]), '4': array([1, 0])}
Incoming:6
Top_dict:{'1': array([1, 0]), '2': array([1, 0]), '3': array([1, 0]), '4': array([1, 0]), '5': array([1, 0])}
Incoming:7
Top_dict:{'2': array([1, 0]), '3': array([1, 0]), '4': array([1, 0]), '5': array([1, 0]), '6': [2, 1]}
Incoming:8
Top_dict:{'3': array([1, 0]), '4': array([1, 0]), '5': array([1, 0]), '6': [2, 1], '7': [2, 1]}
Incoming:9
Top_dict:{'4': array([1, 0]), '5': array([1, 0]), '6': [2, 1], '7': [2, 1], '8': [2, 1]}
Incoming:10
Top_dict:{'5': array([1, 0]), '6': [2, 1], '7': [2, 1], '8': [2, 1], '9': [2, 1]}


In [ ]:
import numpy as np
x=dict()
x['a']=np.array((6,1),dtype='int32')
x['b']=np.array((2,2),dtype='int32')
x['c']=np.array((4,2),dtype='int32')


In [ ]:
min_ele = min(x, key=lambda a: x.get(a)[0])
min_ele